In [1]:
using Pickle, JSON, CSV, DataFrames, BenchmarkTools, Distributed, FileIO, Pickle, Unmarshal
using NetPricing
using JuMP
using Gurobi

In [2]:
# Define a struct to store the results
struct OptimizationResult
	id::AbstractString
	tvals::Vector{Float64}
	obj_value::Float64
	preprocess_time::Float64
	solve_time::Float64
	flow::Dict
	finish::Bool
end


function FindMN(form_type::Type{GeneralFormulation{P,D}}, probs;
    bigM_difference=true,
    kwargs...) where {P,D}

    forms = convert.(Formulation, filter!(!isnothing, assign.(form_type, probs; bigM_difference)))
    # Big M
    Ms = [NetPricing.calculate_bigM(form, threads=nothing) for form in forms]
    N = max.(collect.(maximum.(Ms, dims=2))...)
    return Ms, N #formulate!(forms, linearization; kwargs...)
end

MN(probs; kwargs...) = FindMN(StandardFormulation, probs; kwargs...)
const cst_mn = MN


function projectionN(transformation::Dict, N)
    # println("projectionN")
    # N ---> N~
    # NS : vector of subvectors. The values in subvector at position i
    # are the value from N for every arc in the class [i].
    # Remember that the transformation create a partition of edges that we consider
    # equivalent. The same applies with the N values, NS is a partition of N base on the
    # the edge partition aka the transformation.
    NS = [[] for _ in 1:maximum(values(transformation))]
    for i in keys(transformation)
    	if typeof(i)!=typeof(1)
    		j = parse(Int, i)
    	else
    		j=i
    	end
        append!(
                NS[transformation[i]], N[j]#N[parse(Int, i)]
        )
    end
    
    # Multiple choice (min or max)
    # NT_* : vector representing the maximum value that at least one user is ready to pay
    # in the case that this user have to pass by the edge i. In this case, we borrow the true
    # N from the original problem and adapt to the transform problem.
    # Therefore, the vector could lead to suboptimal objective in the transform problem,
    # but should be valid when we return the original problem.
    # The idea is to solve the smaller problem and use the result as the new N for
    # the original problem with the hope of good time reduction and good approximation.
    
    NT_min = [minimum(vec) for vec in NS]
    NT_avg = [mean(vec) for vec in NS]
    NT_max = [maximum(vec) for vec in NS]
    return NT_min, NT_avg, NT_max
end


function retroprojectionN(transformation::Dict, NT)
    # println("retroprojectionN")
    # N~ ---> N (or t*~ ---> N)
    # N : vector representing the maximum value that at least one user is ready to pay
    # in the case that this user have to pass by the edge i. We reuse the transformation
    # to expand the NT vector that come from the contracted space.
    N = zeros(length(transformation))
    for (k, v) in transformation
    	if typeof(k)!=typeof(1)
    		j = parse(Int, k)
    	else
    		j=k
    	end
        N[j] = NT[v]
    end
    
    return N
end

function projectionM(transformation::Dict, M)
    # println("projectionM")
    MT_min = [[] for _ in 1:length(M)]
	MT_avg = [[] for _ in 1:length(M)]
    MT_max = [[] for _ in 1:length(M)]
    for (i, row) in enumerate(eachrow(M))
        MT_min[i], MT_avg[i], MT_max[i] = projectionN(transformation, first(row))
    end
    return MT_min, MT_avg, MT_max
end

function retroprojectionM(transformation::Dict, MT)
    # println("retroprojectionM")
    M = [[] for _ in 1:length(MT)]
    for (i, row) in enumerate(eachrow(MT))
        M[i] = retroprojectionN(transformation, first(row))
    end
    return M
end

# Save result
# Function to save results to a file
function save_result_individual(result::OptimizationResult, filename::AbstractString)
    open(filename, "w") do file
        JSON.print(file, result)
    end
end

function save_result_batch(results, filename::AbstractString)
	# results : array of OptimizationResult
	# filename : path/to/output.json
	
	# Convert the results list to a JSON string
	json_data = JSON.json(results)

	# Compress the JSON data into a bytes object
	# Save the compressed data to a file
	open(filename, "w") do file
		write(file, json_data)
	end
end

# Import problem
function import_problem_from_file(file::AbstractString)
    # Import a problem from a file
    prob = read_problem(file)
    
	# Get the file name without the extension
	id = splitext(basename(file))[1]
	#println(id)
	return prob, id
end

function import_problem_from_str(str::AbstractString)
	prob = unmarshal(Problem, JSON.parse(str)["problem"])
	return prob
end


function solve_and_get_values(prob::Problem, id::AbstractString, time_limit::Int, M=nothing, N=nothing; option=0)
    try
		# Measure preprocessing time
		# obligatoire
		preprocess_time = @elapsed begin
		    # Preprocess the problem for each commodity
		    pprobs = preprocess(prob, maxpaths = 1000)
		end

		if M!=nothing && N!=nothing
		    println(option)
		    model, forms = NetPricing.cst_model(pprobs, M, N, option=option)
		else
		    # Create a model
		    model, forms = std_model(pprobs)
		end
		
		 # Set GurobiSolver parameters
		set_optimizer(model, Gurobi.Optimizer)
		set_optimizer_attribute(model, "TimeLimit", time_limit) # stop the process after x seconds
		
		# Measure solving time
		solve_time = @elapsed begin
		    # Solve the model
		    optimize!(model)
		end

		# Extract the result
		tvals = value.(model[:t])  # The prices t
		#println(tvals)
		# Get the objective value
		obj_value = objective_value(model)

		flow = []
		for k in 1:length(forms)
			primal_repr = primal(forms[k])              # Primal representation
			prob_k = problem(primal_repr)               # Preprocessed problem of forms[k]
			Amap = used_arcs(prob_k)		    # List of edge index of the solution path 
			#println(Amap)
			append!(flow, Amap)
		end
		
		all_flow = vcat(flow)
		min_val = minimum(all_flow)
		max_val = maximum(all_flow)
		freq_dict = Dict(i => 0 for i in min_val:max_val)

		for val in all_flow
			freq_dict[val] += 1
		end
		
		finish = (solve_time <= time_limit)
		return OptimizationResult(id, tvals, obj_value, preprocess_time, solve_time, freq_dict, finish)
    
    catch
        return nothing
    end
end






function experience(M_original, N_original, transformation::Dict, prob_original::Problem, prob_trans::Problem, id::AbstractString, time_limit::Int)
    
    # Transform Big M, N
    transformation = transformation["TA"]
    NT_min, NT_avg, NT_max = projectionN(transformation, N_original) # minimal, average, maximal projection
    MT_min, MT_avg, MT_max = projectionM(transformation, M_original) # minimal, average, maximal projection

    
    result_trans = solve_and_get_values(prob_trans, id, time_limit);
    result_trans_min = solve_and_get_values(prob_trans, id*"-gamma-min", time_limit , MT_min, NT_min, option=0);
    result_trans_avg = solve_and_get_values(prob_trans, id*"-gamma-avg", time_limit , MT_avg, NT_avg, option=0);
    result_trans_max = solve_and_get_values(prob_trans, id*"-gamma-max", time_limit , MT_max, NT_max, option=0);

   
    N_retro = retroprojectionN(transformation, result_trans.tvals);
    N_retro_min = retroprojectionN(transformation, result_trans_min.tvals);
    N_retro_avg = retroprojectionN(transformation, result_trans_avg.tvals);
    N_retro_max = retroprojectionN(transformation, result_trans_max.tvals);

    # id-0/1-option-min/avg/max
    # 0 : in original space
    # 1 : in transformed space
    # Option 1 - Shortest path
    result_retro_1 = solve_and_get_values(prob_original, id*"-retro-1", 10, M_original, N_retro, option=1);
    # Option 2 - Lower bound
    result_retro_2 = solve_and_get_values(prob_original, id*"-retro-2", time_limit, M_original, N_retro, option=2);
    # Option 3 - Upper bound
    result_retro_3 = solve_and_get_values(prob_original, id*"-retro-3", time_limit, M_original, N_retro, option=3);
    # Option 4 - Comprehensive lower bound
    result_retro_4 = solve_and_get_values(prob_original, id*"-retro-4", time_limit, M_original, N_retro, option=4);
    # Option 5 - Comprehensive lower bound
    result_retro_5 = solve_and_get_values(prob_original, id*"-retro-5", time_limit, M_original, N_retro, option=5);
    
    # Option 1 - Shortest path
    result_retro_1_min = solve_and_get_values(prob_original, id*"-retro-1-min", 10, M_original, N_retro_min, option=1);
    # Option 2 - Lower bound
    result_retro_2_min = solve_and_get_values(prob_original, id*"-retro-2-min", time_limit, M_original, N_retro_min, option=2);
    # Option 3 - Upper bound
    result_retro_3_min = solve_and_get_values(prob_original, id*"-retro-3-min", time_limit, M_original, N_retro_min, option=3);
    # Option 4 - Comprehensive lower bound
    result_retro_4_min = solve_and_get_values(prob_original, id*"-retro-4-min", time_limit, M_original, N_retro_min, option=4);
    # Option 4 - Comprehensive upper bound
    result_retro_5_min = solve_and_get_values(prob_original, id*"-retro-5-min", time_limit, M_original, N_retro_min, option=5);
    
    # Option 1 - Shortest path
    result_retro_1_avg = solve_and_get_values(prob_original, id*"-retro-1-avg", 10, M_original, N_retro_avg, option=1);
    # Option 2 - Lower bound
    result_retro_2_avg = solve_and_get_values(prob_original, id*"-retro-2-avg", time_limit, M_original, N_retro_avg, option=2);
    # Option 3 - Upper bound
    result_retro_3_avg = solve_and_get_values(prob_original, id*"-retro-3-avg", time_limit, M_original, N_retro_avg, option=3);
    # Option 4 - Comprehensive lower bound
    result_retro_4_avg = solve_and_get_values(prob_original, id*"-retro-4-avg", time_limit, M_original, N_retro_avg, option=4);
    # Option 5 - Comprehensive upper bound
    result_retro_5_avg = solve_and_get_values(prob_original, id*"-retro-5-avg", time_limit, M_original, N_retro_avg, option=5);

    # Option 1 - Shortest path
    result_retro_1_max = solve_and_get_values(prob_original, id*"-retro-1-max", 10, M_original, N_retro_max, option=1);
    # Option 2 - Lower bound
    result_retro_2_max = solve_and_get_values(prob_original, id*"-retro-2-max", time_limit, M_original, N_retro_max, option=2);
    # Option 3 - Upper bound
    result_retro_3_max = solve_and_get_values(prob_original, id*"-retro-3-max", time_limit, M_original, N_retro_max, option=3);
    # Option 4 - Comprehensive lower bound
    result_retro_4_max = solve_and_get_values(prob_original, id*"-retro-4-max", time_limit, M_original, N_retro_max, option=4);
    # Option 5 - Comprehensive upper bound
    result_retro_5_max = solve_and_get_values(prob_original, id*"-retro-5-max", time_limit, M_original, N_retro_max, option=5);

    return [
        result_trans, result_trans_min, result_trans_avg, result_trans_max,
        result_retro_1, result_retro_2, result_retro_3, result_retro_4, result_retro_5,
        result_retro_1_min, result_retro_2_min, result_retro_3_min, result_retro_4_min, result_retro_5_min, 
        result_retro_1_avg, result_retro_2_avg, result_retro_3_avg, result_retro_4_avg, result_retro_5_avg, 
        result_retro_1_max, result_retro_2_max, result_retro_3_max, result_retro_4_max, result_retro_5_max
    ]
end


experience (generic function with 1 method)

In [3]:
# file_original = "./../tmp/data/original/000000-000000-d40-08-P.json"
# file_trans = "./../tmp/data/d40-08/000015-100-2-2-1-1-1-1-1-1500-d40-08-P.json"
# file_gamma = "./../tmp/data/d40-08/000015-100-2-2-1-1-1-1-1-1500-d40-08-T.json"


# file_original = "./../tmp/data/original/000000-000000-g30-10-P.json"
# file_trans = "./../tmp/data/g30-10/000048-100-3-3-5-1-1-1-0-1500-g30-10-P.json"
# file_gamma = "../tmp/data/g30-10/000048-100-3-3-5-1-1-1-0-1500-g30-10-T.json"

# file_original = "./../tmp/data/original/000000-000000-g30-10-P.json"
# file_trans = "./../tmp/data/g30-10/000002-100-2-0-1-1-1-1-1-1500-g30-10-P.json"
# file_gamma = "../tmp/data/g30-10/000002-100-2-0-1-1-1-1-1-1500-g30-10-T.json"

file_original = "./../tmp/data/original/000000-000000-d35-02-P.json"
file_trans = "./../tmp/data/d35-02/000069-100-2-0-5-1-1-1-1-1500-d35-02-P.json"
file_gamma = "./../tmp/data/d35-02/000069-100-2-0-5-1-1-1-1-1500-d35-02-T.json"


transformation = JSON.parsefile(file_gamma)

# Import original problem 
prob_original = read_problem(file_original)
pprobs_original = preprocess(prob_original, maxpaths = 1)
# Find Big M, N
M_original, N_original  = cst_mn(pprobs_original);

# Transform Big M, N

# NT_min, NT_max = projectionN(transformation, N_original)
# MT_min, MT_max = projectionM(transformation, M_original)

# M_retro = retroprojectionM(transformation, MT_min)
# N_retro = retroprojectionN(transformation, NT_min)


# Import transform problem
prob_trans = read_problem(file_trans);
#pprobs_trans = preprocess(prob_trans, maxpaths = 1000);

In [4]:
id_original = "000000-000000-d35-02"
id_trans = "000069-100-2-0-5-1-1-1-1-1500-d35-02"
time = 30

result_original = solve_and_get_values(prob_original, id_original, time);
res = experience(M_original, N_original,transformation, prob_original, prob_trans, id_trans, time )
result_trans, result_trans_min, result_trans_avg, result_trans_max,  result_retro_1, result_retro_2, result_retro_3, result_retro_4, result_retro_5, result_retro_1_min, result_retro_2_min, result_retro_3_min, result_retro_4_min, result_retro_5_min, result_retro_1_avg, result_retro_2_avg, result_retro_3_avg, result_retro_4_avg, result_retro_5_avg, result_retro_1_max, result_retro_2_max, result_retro_3_max, result_retro_4_max, result_retro_5_max = res


Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-10
Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-10
Set parameter TimeLimit to value 30
Set parameter TimeLimit to value 30
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 24.04 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i5-1340P, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 13264 rows, 10199 columns and 47293 nonzeros
Model fingerprint: 0x72b97454
Variable types: 8602 continuous, 1597 integer (1597 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e+00, 2e+04]
  Bounds range     [1e+00, 9e+01]
  RHS range        [1e-10, 2e+02]
Presolve removed 2420 rows and 1741 columns
Presolve time: 0.12s
Presolved: 10844 rows, 8458 columns, 43150 nonzeros
Variable types: 6842 continuous, 1616 integer (1616 binary)

Root relax

24-element Vector{Union{Nothing, OptimizationResult}}:
 OptimizationResult("000069-100-2-0-5-1-1-1-1-1500-d35-02", [21.0, 34.0, 51.0, 25.0, 28.0, 1.0, 0.0, 12.0, 0.0, 0.0  …  20.0, 5.0, 1.0, 0.0, 5.0, 20.0, 0.0, 16.0, 13.0, 19.0], 158094.0, 2.271875599, 30.060723373, Dict(719 => 5, 699 => 0, 673 => 1, 73 => 3, 319 => 0, 251 => 0, 687 => 4, 115 => 0, 112 => 1, 185 => 8…), false)
 OptimizationResult("000069-100-2-0-5-1-1-1-1-1500-d35-02-gamma-min", [21.0, 33.0, 51.0, 24.0, 28.0, 41.0, 26.0, 12.0, 0.0, 0.0  …  19.0, 48.0, 1.0, 42.0, 15.0, 20.0, 15.856617757082034, 16.0, 13.0, 19.0], 157880.0, 2.913987724, 30.051101362, Dict(719 => 5, 699 => 0, 673 => 1, 73 => 3, 319 => 0, 251 => 0, 687 => 4, 115 => 0, 112 => 1, 185 => 8…), false)
 OptimizationResult("000069-100-2-0-5-1-1-1-1-1500-d35-02-gamma-avg", [21.0, 34.0, 51.0, 25.0, 28.0, 41.0, 26.0, 12.0, 0.0, 0.0  …  26.0, 0.0, 1.0, 0.0, 15.0, 20.0, 60.0, 16.0, 13.0, 19.0], 157880.0, 3.298204423, 30.051319648, Dict(719 => 5, 699 => 0, 673 => 1, 7

In [10]:
println("Trans. optimal    ", result_trans.obj_value, "\t", result_trans.solve_time)
println("Trans.min optimal ", result_trans_min.obj_value, "\t", result_trans_min.solve_time)
println("Trans.avg optimal ", result_trans_avg.obj_value, "\t", result_trans_avg.solve_time)
println("Trans.max optimal ", result_trans_max.obj_value, "\t", result_trans_max.solve_time)
println("True optimal      ", result_original.obj_value, "\t", result_original.solve_time)
println()
println("Experience        ", "Objective",            "\t", "retro solving time")
# Without custom M, N in the transform problem 
# println("retro 1           ", result_retro_1.obj_value, "\t", result_retro_1.solve_time)#, "\t", result_trans.solve_time)  
# println("retro 2           ", result_retro_2.obj_value, "\t", result_retro_2.solve_time)#, "\t", result_trans.solve_time)  
println("retro 3           ", result_retro_3.obj_value, "\t", result_retro_3.solve_time)#, "\t", result_trans.solve_time)  
println("retro 4           ", result_retro_4.obj_value, "\t", result_retro_4.solve_time)#, "\t", result_trans.solve_time)  
println("retro 5           ", result_retro_5.obj_value, "\t", result_retro_5.solve_time)#, "\t", result_trans.solve_time)  
println()
# With custom M, N in the transform problem (min)
println("retro 1 min      ", result_retro_1_min.obj_value, "\t", result_retro_1_min.solve_time)#, "\t", result_trans_min_0.solve_time) # 
println("retro 2 min      ", result_retro_2_min.obj_value, "\t", result_retro_2_min.solve_time)#, "\t", result_trans_min_0.solve_time) # 
println("retro 3 min      ", result_retro_3_min.obj_value, "\t", result_retro_3_min.solve_time)#, "\t", result_trans_min_0.solve_time) # 
println("retro 4 min      ", result_retro_4_min.obj_value, "\t", result_retro_4_min.solve_time)#, "\t", result_trans_min_0.solve_time) # 
println("retro 5 min      ", result_retro_5_min.obj_value, "\t", result_retro_5_min.solve_time)#, "\t", result_trans_min_0.solve_time) # 
println()
# With custom M, N in the transform problem (avg)
# println("retro 1 avg      ", result_retro_1_avg.obj_value, "\t", result_retro_1_avg.solve_time)#, "\t", result_trans_min_0.solve_time) # 
# println("retro 2 avg      ", result_retro_2_avg.obj_value, "\t", result_retro_2_avg.solve_time)#, "\t", result_trans_min_0.solve_time) # 
println("retro 3 avg      ", result_retro_3_avg.obj_value, "\t", result_retro_3_avg.solve_time)#, "\t", result_trans_min_0.solve_time) # 
println("retro 4 avg      ", result_retro_4_avg.obj_value, "\t", result_retro_4_avg.solve_time)#, "\t", result_trans_min_0.solve_time) # 
println("retro 5 avg      ", result_retro_5_avg.obj_value, "\t", result_retro_5_avg.solve_time)#, "\t", result_trans_min_0.solve_time) # 
println()
# With custom M, N in the transform problem (min)
# println("retro 1 max      ", result_retro_1_max.obj_value, "\t", result_retro_1_max.solve_time)#, "\t", result_trans_max_0.solve_time) # 
# println("retro 2 max      ", result_retro_2_max.obj_value, "\t", result_retro_2_max.solve_time)#, "\t", result_trans_max_0.solve_time) # 
println("retro 3 max      ", result_retro_3_max.obj_value, "\t", result_retro_3_max.solve_time)#, "\t", result_trans_max_0.solve_time) # 
println("retro 4 max      ", result_retro_4_max.obj_value, "\t", result_retro_4_max.solve_time)#, "\t", result_trans_max_0.solve_time) #  
println("retro 5 max      ", result_retro_5_max.obj_value, "\t", result_retro_5_max.solve_time)#, "\t", result_trans_max_0.solve_time) #  



Trans. optimal    158094.0	30.060723373
Trans.min optimal 157880.0	30.051101362
Trans.avg optimal 157880.0	30.051319648
Trans.max optimal 157880.0	30.064101495
True optimal      164584.00000008088	30.985712857

Experience        Objective	retro solving time
retro 3           153904.0	10.423726891
retro 4           154031.0	13.460153686
retro 5           153904.0	12.848365047

retro 1 min      119338.0	0.301426142
retro 2 min      126237.0	3.489116162
retro 3 min      158591.0	3.520564318
retro 4 min      155916.35030977512	11.710671697
retro 5 min      158591.0	3.349926517

retro 3 avg      156338.0	3.38745817
retro 4 avg      156341.0	16.118588384
retro 5 avg      156338.0	2.88072396

retro 3 max      154127.0	5.138297102
retro 4 max      155480.0	17.461878191
retro 5 max      154127.0	8.799416054


In [11]:
# d40-08
######################################################
((164584-155916)/164584)*100
# h50-04
######################################################

# h50-04
######################################################

5.266611578282214

In [1]:
x = [10, 20, 30, 40]
random_vector = [rand(1:x_i) for x_i in x]

4-element Vector{Int64}:
  1
  2
 23
 21

In [ ]:
julia ./src/julia/script.jl ./tmp/data/d40-08/000011-100-2-2-1-1-1-1-1-1500-d40-08-P.json ./tmp/data/d40-08/000011-100-2-2-1-1-1-1-1-1500-d40-08-R.json 25 ./tmp/data/000000-000000-d40-08-P.json ./tmp/data/d40-08/000011-100-2-2-1-1-1-1-1-1500-d40-08-T.json